## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from Config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [25]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Desc
0,0,Ingraj Bazar,IN,25.0000,88.1500,83.50,78,39,6.29,scattered clouds
1,1,Atar,MR,20.5169,-13.0499,103.26,25,61,4.21,broken clouds
2,2,Cape Town,ZA,-33.9258,18.4232,61.21,64,40,5.99,scattered clouds
3,3,Nouadhibou,MR,20.9310,-17.0347,93.18,46,0,8.05,clear sky
4,4,Bambous Virieux,MU,-20.3428,57.7575,72.21,73,5,5.32,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = int(input('What is your minimum temperature?'))
max_temp = int(input('What is your maximum temperature?'))

What is your minimum temperature?70
What is your maximum temperature?80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)] 
                                    
filter_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Desc
4,4,Bambous Virieux,MU,-20.3428,57.7575,72.21,73,5,5.32,clear sky
11,11,Kloulklubed,PW,7.0419,134.2556,77.50,71,100,5.61,overcast clouds
16,16,Boende,CD,-0.2167,20.8667,74.77,76,100,2.48,overcast clouds
22,22,Dumai,ID,1.6833,101.4500,74.61,92,100,2.84,overcast clouds
23,23,Kapaa,US,22.0752,-159.3190,75.52,81,90,9.22,light rain
27,27,Hasaki,JP,35.7333,140.8333,77.40,85,100,12.53,heavy intensity rain
28,28,Yokadouma,CM,3.5167,15.0500,75.56,82,97,2.08,light rain
34,34,Marathon,US,44.9000,-89.7668,74.73,86,90,1.01,overcast clouds
44,44,Puerto Ayora,EC,-0.7393,-90.3518,71.56,96,95,5.01,light rain
45,45,Tezu,IN,27.9167,96.1667,76.39,98,98,2.48,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
filter_df.count()

City_ID         265
City            265
Country         265
Lat             265
Lng             265
Max Temp        265
Humidity        265
Cloudiness      265
Wind Speed      265
Current Desc    265
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
full_df = filter_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = full_df[['City', 'Country', 'Max Temp', 'Current Desc', 'Lat', 'Lng']].copy()

# 5b. Create a new column "Hotel Name".
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Current Desc,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,72.21,clear sky,-20.3428,57.7575,
11,Kloulklubed,PW,77.50,overcast clouds,7.0419,134.2556,
16,Boende,CD,74.77,overcast clouds,-0.2167,20.8667,
22,Dumai,ID,74.61,overcast clouds,1.6833,101.4500,
23,Kapaa,US,75.52,light rain,22.0752,-159.3190,
27,Hasaki,JP,77.40,heavy intensity rain,35.7333,140.8333,
28,Yokadouma,CM,75.56,light rain,3.5167,15.0500,
34,Marathon,US,74.73,overcast clouds,44.9000,-89.7668,
44,Puerto Ayora,EC,71.56,light rain,-0.7393,-90.3518,
45,Tezu,IN,76.39,overcast clouds,27.9167,96.1667,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat}, {lng}'

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        pass

In [15]:
# 7. Drop the rows where there is no Hotel Name.
completed_df = hotel_df[hotel_df['Hotel Name']!= '']
completed_df

,City,Country,Max Temp,Current Desc,Lat,Lng,Hotel Name
4,Bambous Virieux,MU,72.21,clear sky,-20.3428,57.7575,Casa Tia Villa
11,Kloulklubed,PW,77.50,overcast clouds,7.0419,134.2556,Storyboard Beach Resort
22,Dumai,ID,74.61,overcast clouds,1.6833,101.4500,Grand Zuri Dumai
23,Kapaa,US,75.52,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
27,Hasaki,JP,77.40,heavy intensity rain,35.7333,140.8333,Hotel Sunrise Choshi
...,...,...,...,...,...,...,...
1298,Anloga,GH,75.24,light rain,5.7947,0.8973,Pin Drop Hotel
1300,Concarneau,FR,70.05,few clouds,47.8750,-3.9225,Brit Hotel de L' Océan
1303,Ilheus,BR,79.03,clear sky,-14.7889,-39.0494,Barravento Hotel
1307,Puerto Ayora,EC,71.56,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel


In [17]:
# 8a. Create the output File (CSV)
output_data_file ='WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
completed_df.to_csv(output_data_file, index_label='City_ID')

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Desc} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in completed_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = completed_df[["Lat", "Lng"]]

In [24]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))